In [1]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

········


In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

········


In [6]:
from langchain_core.messages import HumanMessage

# Calling the model in this way will not allow it to keep any context from previous messages... would need to pass
# in each prior response from both user and AI in order to get this 
# Like this:
# model.invoke(
#     [
#         HumanMessage(content="Hi! I'm Bob"),
#         AIMessage(content="Hello Bob! How can I assist you today?"),
#         HumanMessage(content="What's my name?"),
#     ]
# )

# But this would not make sense for a longer conversation obviously

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a578d200-795e-4d73-a004-0fb5531585cc-0')

In [10]:
pip install langchain_community

     |████████████████████████████████| 2.1 MB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [12]:
config = {"configurable": {"session_id": "abc2"}} #use this same session ID to maintain the history in diff calls
# Using a diff session ID would change the history that the bot uses, but can always go back to ones you've used
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [15]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

In [16]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

response = with_message_history.invoke(
    [HumanMessage(content="What's the first letter of my name?")],
    config=config,
)

response.content

'The first letter of your name is "B".'

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [20]:
# Have to specify which key to use to save the chat history because there are multiple now
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Lo siento, no tengo la capacidad de saber tu nombre. ¿En qué más te puedo ayudar?'

In [21]:
# Have to manage convo history when doing this because otherwise it can overflow context window of LLM - do this
# by limiting size of meessages you pass
# NEED TO DO THIS BEFORE PROMPT TEMPLATE BUT AFTER LOAD PREVIOUS MESSAGES FROM MESSAGE HISTORY

from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [23]:
from langchain_core.messages import AIMessage

# Now can see that if we do more than 10 messages, it won't remember the earlier ones
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to that information. How can I assist you today?"

In [26]:
# Can easily add streaming (returning words of response individually as they are generated) - very crucial for UX

config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="")

Hi Todd! Here's another joke for you:

What do you call an alligator in a vest?

An investigator! 😄